In [3]:
import pandas as pd
import re

from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from konlpy.tag import Hannanum, Kkma, Komoran, Okt
import time

data = pd.read_pickle("../../petition/petition_total_data.pkl")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40054 entries, 0 to 40053
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   code      40054 non-null  object        
 1   sdays     40054 non-null  datetime64[ns]
 2   edays     40054 non-null  datetime64[ns]
 3   title     40054 non-null  object        
 4   count     40054 non-null  int64         
 5   content   40054 non-null  object        
 6   category  40054 non-null  object        
 7   progress  40054 non-null  object        
 8   link      40054 non-null  object        
 9   person    40054 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 3.1+ MB


In [5]:
data['progress'] = [re.sub("\n",'',progress) for progress in data['progress']]

In [6]:
data = data.reset_index(drop=True)
data.shape

(40054, 10)

In [7]:
data['sdays'] = pd.to_datetime(data['sdays'])
data['edays'] = pd.to_datetime(data['edays'])

data['count'] = pd.to_numeric(data['count'])

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

data['content'] = [re.sub('|'.join(['\n','\t','\r']),'',''.join(content)).strip() for content in data['content']]

data['content'] = [emoji_pattern.sub(r'', content) for content in data['content']]

In [8]:
data['month'] = data['sdays'].dt.month
data_12 = data[data['month']==12]

In [9]:
data_12 = data_12.reset_index(drop=True)

In [10]:
data_12 = data[data['month']==12]

kkma= Kkma()

doc_nouns_list = [' '.join(kkma.nouns(doc)) for doc in data_12['content']]

print(doc_nouns_list)

['저 11 11월 월 4 4일 일 어린이 어린이집 집 아파트 단지 단지내 내 동갑내기 남자 남자아이 아이 친구 앞 바지 항문과 성기 손가락 아동 아동성폭력 성폭력 피해 만 만5세 5 세 딸 딸아이 아버지 아동청소년 청소년 성 성보호 보호 법제 법제2조 2 조 정의 1 아동ㆍ청소년 ㆍ 19 19세 미만 자 도달 연도 1월 1일 자 제외 아동ㆍ청소년대상 대상 성범죄 다음 목 하나 해당 죄 7 7조 조 15 15조 만 5세 법 강간 강간ㆍ강제추행 강제 추행 등 폭 폭행 행 협박 아동ㆍ청소년 사람 무기 무기징역 징역 5년 년 이상 유기 유기징역 호의 행위 구강 구강ㆍ항문 항문 신체 의 내부 행위2 성기ㆍ항문 일부 도구 자전거 보관소 반 이 6 질 진물 입 쉬 파 똥 말 피해자 성범죄자 범죄자 아동복 아동복지법 지법 17 17조 금지 금지행위 누구 개정 2014 28 매매 음란 매개 성적 수치심 성희롱 학대 학대행위3 3 손상 건강 발달 신체적 학대행위아동복지법 학대행위 6살 살 형법 9 9조 형사 성년자 14 14세 세 자의 벌 형사미성년 미성년 뿐 유죄 무죄 수 상 처벌 처벌대상 표현 형사처벌 처음 고소 고소접수 접수 현실 저희 사례 가정 절망감 법제41조 41 피해아동ㆍ청소년 조치 청구 검사 지속적 위해 배제 필요 필요하다 하다 인정 경우 법원 1호 호 보호관찰 관찰 2호 5호 특정 전자 전자장치 장치 부착 법률 1항 항 3호 가해자 특정지역 지역 출입 출입금지 준수 준수사항 사항 부과 보호관찰2 주거 분리 퇴거 조치3 학교 100 100미터 미터 이내 대리인 접근 아파트단지 당한 트라 트라우마 우마 주차장 라 곳 공포 밤 악몽 마 안해 잠꼬대 연일 거주 거주지로 지로 분리조치 옆 부모 자기 자식 취급 말랍니 이사 그것 자기 자기동 동 우리 우리동 통로 차가 주차 피 266 266조 과실 과실치상 치상 상해 500 500만원 원 이하 벌금 구류 과료 1995.12 29 명시 의사 공소 제기 민법 755 755조 감독자 책임 손해 753 753조 조 754 754

In [11]:

tfidf_vectorizer = TfidfVectorizer(min_df=2)  # 한 번만 나타나는 단어들은 무시
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)


vector_array = tfidf_matrix.toarray()
nmf = NMF(n_components=45, random_state = 0)
nmf.fit(vector_array)
features = nmf.transform(vector_array)

normalizer = Normalizer()
norm_features=normalizer.fit_transform(features)

print(norm_features[0:2])

df_features = pd.DataFrame(norm_features,index=data_12['title'].tolist())


C:\Users\a0105\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


[[0.00642497 0.         0.         0.         0.         0.37337369
  0.         0.01520203 0.02178141 0.01632104 0.16133021 0.
  0.0418467  0.         0.1108409  0.11505832 0.         0.
  0.03273426 0.01491619 0.20802253 0.         0.08060195 0.
  0.         0.         0.03993871 0.         0.00247772 0.04089299
  0.19193373 0.         0.         0.14497275 0.67470951 0.03968601
  0.22277356 0.05735183 0.42498662 0.         0.         0.
  0.         0.02882703 0.05397376]
 [0.         0.         0.02786187 0.00304707 0.08752057 0.
  0.06953329 0.         0.         0.         0.         0.
  0.         0.         0.         0.25267187 0.         0.03269029
  0.         0.         0.01937882 0.         0.01065486 0.
  0.         0.21502643 0.         0.00699796 0.         0.
  0.         0.05389732 0.         0.         0.         0.
  0.         0.77963279 0.00836751 0.         0.         0.15561538
  0.48991346 0.         0.        ]]


In [12]:
df_features.index

Index(['아동간 성폭력사고 시 강제력을 가진 제도를 마련해주시기 바랍니다.',
       '한국기독교총연합회 사단법인 해산과  전** 대표회장 구속을 촉구 합니다',
       '코로나전쟁에 왜 자영업자만 일방적 총알받이가 되나요? 대출원리금 임대료 같이 멈춰야 합니다.',
       '잔인하고도 무서운 학교폭력으로 우리아들의 인생이 망가졌습니다.', '불법도박의 피해가 점차 심각해지고 있습니다 도와주세요!!',
       '국회의원 세비 인상 반대합니다', '살인마들 다단계코인 사기범', '어린이 보육시설 cctv 상시 공개 의무화',
       '**고등학교의 횡포를 멈춰주세요!!', '인공관절 수술후 식물인간으로계시다가 돌아가셨습니다',
       ...
       '서울마포 3700세대 아파트에서 발암물질 페놀온수가 나옵니다', '전광훈을 무죄판결한 허선아 판사 탄핵 청원합니다.',
       '인천 서구 국공립 어린이집 장애 아동 집단 학대 사건', 'ai 조류독감 무자비한 살처분 정책을 막아주세요',
       '허선아 재판부 탄핵을 촉구합니다.', '****의 신용점수제는 대국민 사기극!',
       '서울 마을버스 업체 지원 정부에서 해결해주세요', '과거사위원회를 상설기구로 설치하여 주십시오.',
       '정부와 보건복지부에 요청 합니다 의대생 국시 절대 반대 합니다',
       '내년 의사 국가고시를 1월에 추가로 실시하면서 정부를 믿고 올해 응시한 423명을 배신해 놓고, 그들을 위해서는 아무런 말도 하지 않았습니다. 여기에 대한 대책을 요구합니다.'],
      dtype='object', length=1348)

In [13]:
article = df_features.loc['코로나전쟁에 왜 자영업자만 일방적 총알받이가 되나요? 대출원리금 임대료 같이 멈춰야 합니다.']
similarities = df_features.dot(article)
top = similarities.nlargest(5)

for i in range(1, len(top)):
    print("Title: "+top.index[i]+'\n Similarities'+str(top[i]))

Title: 3주 집합금지 업체 임대료 삭감을 원합니다.
 Similarities0.9273903339206672
Title: 재난지원금 지급의 형평성 문제
 Similarities0.9264007811087569
Title: 12월8일부로 명령된 코로나 집합금지 대상 업종 임대료 의무 삭감 요청
 Similarities0.9236974887227365
Title: 자영업자들 정말 죽습니다 임대료 또한 일정한 비율로 삭감할 수 있도록 도와주세요
 Similarities0.8940252569183683


In [14]:
article = df_features.loc['아동간 성폭력사고 시 강제력을 가진 제도를 마련해주시기 바랍니다.']
similarities = df_features.dot(article)
top = similarities.nlargest()

for i in range(1, len(top)):
    print("Title: "+top.index[i]+'\n Similarities'+str(top[i]))

Title: 억울한 누명으로 20년의 세월을 송두리째 빼앗긴 윤 씨에게 보상해주기를 청원합니다.
 Similarities0.7872509089631238
Title: 딸의 남자친구가 제 딸과, 언니인 제 큰 딸까지 살해하였습니다.
 Similarities0.7724757052712977
Title: 피해자 중심적인 보도와 법률집행의 폐단을 없애주세요
 Similarities0.7703202971323151
Title: 30인분 장난주문한 닭강정 사건 엄청수사와 반드시 구속 시킵시다
 Similarities0.7546010710525259


In [15]:
article = df_features.loc['정부와 보건복지부에 요청 합니다 의대생 국시 절대 반대 합니다']
similarities = df_features.dot(article)
top = similarities.nlargest()

for i in range(1, len(top)):
    print("Title: "+top.index[i]+'\n Similarities'+str(top[i]))

Title: 의대생 국시 재시험은 0.1%의 가능성도 없어야 합니다.
 Similarities0.9943622152249144
Title: 의사국시 재시험 반대합니다.
 Similarities0.9928033428180237
Title: ‘자의로’ 시험을 거부한 의대생의 구제를 반대합니다
 Similarities0.9925832910334086
Title: 정세균 국무총리 사퇴해주세요
 Similarities0.9915596384912423


In [ ]:
#  Komoran이 okt보다 더 좋은 결과.